# Fast Implied Volatilities using the NAG Library

The Black-Scholes formula for the price of a European call option is

$$P = S_0\Phi\left(\frac{\ln\left(\frac{S_0}{K}\right)+\left[r+\frac{\sigma^2}{2}\right]T}{\sigma\sqrt{T}}\right) - Ke^{-rT}\Phi\left(\frac{\ln\left(\frac{S_0}{K}\right)+\left[r-\frac{\sigma^2}{2}\right]T}{\sigma\sqrt{T}}\right),$$

where $T$ is the time to maturity, $S_0$ is the spot price of the underlying asset, $K$ is the strike price, $r$ is
the interest rate and $\sigma$ is the volatility. A similar formula applies for European put options.

An important problem in finance is to compute the implied volatility, $σ$, given values for $T$, $K$, $S_0$,
$r$ and $P$. An explicit formula for $\sigma$ is not available. Furthermore, $\sigma$ cannot be directly measured from
financial data. Instead, it must be computed using a numerical approximation. Typically, multiple values
of the input data are provided, so the Black-Scholes formula must be solved many times.

As shown in the figure below, the volatility surface (a three-dimensional plot of how the volatility varies
according to the price and time to maturity) can be highly curved. This makes accurately computing
the implied volatility a difficult problem.

<img src="impvolsurf.png" width=500 />

Before introducing our new NAG Library routine, let’s demonstrate how one might naively
compute implied volatilities using a general purpose root finder. First we need to import a few things:

In [1]:
import numpy as np
import time
from naginterfaces.library import rand, roots, specfun

Let's generate some input data using a random number generator from the NAG Library:

In [2]:
n = 10000 # This is the number of volatilities we will be computing
statecomm = rand.init_nonrepeat(1)
p = rand.dist_uniform(n, 3.9, 5.8, statecomm)
k = rand.dist_uniform(n, 271.5, 272.0, statecomm)
s0 = rand.dist_uniform(n, 259.0, 271.0, statecomm)
t = rand.dist_uniform(n, 0.016, 0.017, statecomm)
r = rand.dist_uniform(n, 0.017, 0.018, statecomm)

We have chosen the limits of the various uniform distributions above to ensure the input data takes
sensible values.

There are various standard root finding techniques that we could use to compute implied volatilities,
a common example being bisection. The NAG Library routine ```contfn_cntin```, is a general
purpose root finder based on the secant method. It uses a *callback*, with data passed in via a communication object:

In [3]:
def black_scholes(sigma, data):
    try:
        price = specfun.opt_bsm_price('C', [data['k']], data['s0'], [data['t']], sigma, data['r'], 0.0)
    except:
        price = np.zeros((1,1))
    return price[0, 0] - data['p']

 ```contfn_cntin``` operates on scalars, so we need to call the routine
once for every volatility we want to compute. We will time the computation and count how many errors are caught:

In [4]:
data = {}
errorcount = 0
tic = time.perf_counter()
for i in range(n):
    data['p'] = p[i]
    data['k'] = k[i]
    data['s0'] = s0[i]
    data['t'] = t[i]
    data['r'] = r[i]
    try:
        sigma = roots.contfn_cntin(0.15, black_scholes, 1.0e-14, 0.0, 500, data)
        if sigma < 0.0:
            errorcount += 1
    except:
        errorcount += 1
toc = time.perf_counter()      
print('Using a general purpose root finder:')
print('    Time taken: {0:.3f} seconds'.format(toc-tic))
print('    There were {} failures'.format(errorcount))

Using a general purpose root finder:
    Time taken: 51.561 seconds
    There were 4 failures


Can a bespoke implied volatility routine do better? Our new routine at Mark 27.1 is called ```opt_imp_vol```. We call it as follows:

```sigma, ivalid = specfun.opt_imp_vol('C', p, k, s0, t, r, mode=2)```

The return argument ```ivalid``` is an array recording any data points for which the volatility could not be computed. The argument ```mode``` allows us to select which algorithm to use – more on that in a moment, but
for now we choose ```mode=2```. This selects the algorithm of Jäckel (2015), a very accurate method based
on third order Householder iterations.

Here is the call surrounded by some timing code:

In [5]:
tic = time.perf_counter()
sigma, ivalid = specfun.opt_imp_vol('C', p, k, s0, t, r, mode=2)
toc = time.perf_counter()
print('omp_imp_vol with mode = 2 (Jäckel algorithm):')
print('    Time taken: {0:.5f} seconds'.format(toc-tic))
print('    There were {} failures'.format(np.count_nonzero(ivalid)))

omp_imp_vol with mode = 2 (Jäckel algorithm):
    Time taken: 0.01415 seconds
    There were 0 failures


The new routine is several orders of magnitude faster than the root finder, with no failures reported. We could try
tweaking the convergence parameters and iteration limits in ```nag_roots_contfn_cntin```, and we could certainly
improve the way data is passed through the callback, but we are unlikely to match the
performance of ```opt_imp_vol```.

Recently NAG embarked upon a collaboration with mathematicians at Queen Mary University of
London, who have been developing an alternative algorithm for computing implied volatilities. The new
algorithm (based on Glau et. al. (2018)) uses Chebyshev interpolation to remove branching and give
increased SIMD performance. We access it by setting ```mode=1``` in the call to ```opt_imp_vol```:

In [6]:
tic = time.perf_counter()
sigma, ivalid = specfun.opt_imp_vol('C', p, k, s0, t, r, mode=1)
toc = time.perf_counter()
print('omp_imp_vol with mode = 1 (Glau algorithm):')
print('    Time taken: {0:.5f} seconds'.format(toc-tic))
print('    There were {} failures'.format(np.count_nonzero(ivalid)))

omp_imp_vol with mode = 1 (Glau algorithm):
    Time taken: 0.01770 seconds
    There were 0 failures


Depending on your system, you should find that, for similar accuracy, there is a modest speedup over the Jäckel algorithm. Our numerical experiments have shown that for very small arrays (containing fewer than 100 elements) the Jäckel algorithm actually
outperforms that of Glau et.al., but for larger arrays the converse is true. As vector units continue
to improve in the future, we expect the performance of the highly vectorizable Glau et.al. approach to
improve similarly.

So far, we have been computing implied volatilities with a relative accuracy as close as possible to
double precision. However, in some applications implied volatilities are only required with a few decimal
places of precision. One advantage of the Glau et.al. algorithm is that it can be run in a lower accuracy
mode, aiming only for seven decimal places of accuracy. This is accessed by setting ```mode=0``` in the call
to ```opt_imp_vol```. It roughly doubles the speed of the routine:

In [7]:
tic = time.perf_counter()
sigma, ivalid = specfun.opt_imp_vol('C', p, k, s0, t, r, mode=0)
toc = time.perf_counter()
print('omp_imp_vol with mode = 0 (lower accuracy Glau algorithm):')
print('    Time taken: {0:.5f} seconds'.format(toc-tic))
print('    There were {} failures'.format(np.count_nonzero(ivalid)))

omp_imp_vol with mode = 0 (lower accuracy Glau algorithm):
    Time taken: 0.01556 seconds
    There were 0 failures


The charts below summarize the results, using timings collected on an Intel Skylake machine. We can see that the Glau et.al. algorithm outperforms the Jäckel algorithm for large arrays but not for small arrays. Note that the general purpose root finder
is omitted here as it is so much slower ```opt_imp_vol```.
<img src="graphs.PNG" width=800 />

In summary, NAG’s new state-of-the art algorithm can be run in three different modes, according to
the length of the input arrays and the required accuracy. For more information, and to access the NAG
Library, go to: https://www.nag.co.uk/content/nag-library.

### References

P. Jäckel (2015). Let’s be rational. *Wilmott* 2015, 40-53.

K. Glau, P. Herold, D. B. Madan, C. Pötz (2019). The Chebyshev method for the implied volatility.
*Journal of Computational Finance*, 23(3).

### NAG Library for Python Setup

Find instructions to install the NAG Library for Python in the documentation here: https://www.nag.com/numeric/py/nagdoc_latest/readme.html#installation